# Common imports & library functions

In [8]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
from itertools import count
import math
import re
from copy import deepcopy

# Day 11: Seating System

In [7]:
def _floors(n):
    return ['.'] * n

class SeatLayout:
    def __init__(self, initial_state, neighbor_threshold=4):
        self._neighbor_threshold = neighbor_threshold
        initial_state = initial_state.strip().replace(' ', '')
        cells = [list(row.strip()) for row in initial_state.split()]
        w = len(cells[0])
        self._cells = ([_floors(w + 2)] +
                       [_floors(1) + row + _floors(1) for row in cells] +
                       [_floors(w + 2)])

    def at(self, x, y):
        return self._cells[y+1][x+1]

    def set(self, x, y, state):
        self._cells[y+1][x+1] = state

    def cells(self):
        for y in range(1, len(self._cells) - 1):
            for x in range(1, len(self._cells[0]) - 1):
                yield (x - 1, y - 1, self._cells[y][x])

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                yield self.at(x+dx, y+dy)

    def update(self):
        prev_layout = deepcopy(self)
        updated = False
        for x, y, cell in prev_layout.cells():
            neighbors = list(prev_layout.neighbors(x, y))
            if (cell == 'L' and not any(n == '#' for n in neighbors)):
                self.set(x, y, '#')
                updated = True
            elif (cell == '#' and neighbors.count('#') >= self._neighbor_threshold):
                self.set(x, y, 'L')
                updated = True
        return updated

    def num_occupied(self):
        return sum(r.count('#') for r in self._cells)

    def __str__(self):
        return '\n'.join(''.join(r) for r in self._cells)

class SeatLayout2(SeatLayout):
    def __init__(self, initial_state, neighbor_threshold=5):
        super().__init__(initial_state, neighbor_threshold)

    def within_grid(self, x, y):
        return 0 <= y < len(self._cells) - 1 and 0 <= x < len(self._cells[0]) - 1

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                for xp, yp in zip(count(x+dx, dx), count(y+dy, dy)):
                    if not self.within_grid(xp, yp):
                        break
                    if (cell := self.at(xp, yp)) != '.':
                        yield cell
                        break

def simulate_until_equilibrium(layout):
    while layout.update():
        continue
    return layout.num_occupied()

In [8]:
test_layout = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

assert simulate_until_equilibrium(SeatLayout(test_layout)) == 37
assert simulate_until_equilibrium(SeatLayout2(test_layout)) == 26

In [57]:
# Final answers
with open('day11.txt') as f:
    initial_state = f.read()
    print('Part 1: ', simulate_until_equilibrium(SeatLayout(initial_state)))
    print('Part 2: ', simulate_until_equilibrium(SeatLayout2(initial_state)))

Part 1:  2418
Part 2:  2144


# Day 12: Rain Risk

In [6]:
import math

def move_along_angle(x, y, distance, heading):
    """
    >>> move_along_angle(0, 0, 10, 90)
    (0, 10)
    >>> move_along_angle(4, 1 + 3 * math.sqrt(3), 6, 240)
    (1, 1)
    """
    angle = math.radians(heading)
    x += distance * math.cos(angle)
    y += distance * math.sin(angle)
    return int(round(x)), int(round(y))

def parse_actions(actions):
    for action in actions.strip().split():
        action = action.strip()
        yield action[0], int(action[1:])

def move(actions, start_pos=(0, 0)):
    """
    >>> move('''
    ...     F10
    ...     N3
    ...     F7
    ...     R90
    ...     F11
    ... ''')
    (17, -8)
    """
    x, y = start_pos
    heading = 0  # degrees
    for action, amount in parse_actions(actions):
        if action == 'N':
            y += amount
        elif action == 'S':
            y -= amount
        elif action == 'E':
            x += amount
        elif action == 'W':
            x -= amount
        elif action == 'L':
            heading = (heading + amount) % 360
        elif action == 'R':
            heading = (heading - amount) % 360
        elif action == 'F':
            x, y = move_along_angle(x, y, amount, heading)
    return x, y

In [7]:
doctest.run_docstring_examples(move_along_angle, globs=None, verbose=True)
doctest.run_docstring_examples(move, globs=None, verbose=True)

Finding tests in NoName
Trying:
    move_along_angle(0, 0, 10, 90)
Expecting:
    (0, 10)
ok
Trying:
    move_along_angle(4, 1 + 3 * math.sqrt(3), 6, 240)
Expecting:
    (1, 1)
ok
Finding tests in NoName
Trying:
    move('''
        F10
        N3
        F7
        R90
        F11
    ''')
Expecting:
    (17, -8)
ok


In [8]:
def rotate(x, y, angle):
    """
    >>> rotate(10, 4, -90)
    (4, -10)
    """
    angle = math.radians(angle)
    xp = x * math.cos(angle) - y * math.sin(angle)
    yp = x * math.sin(angle) + y * math.cos(angle)
    return int(round(xp)), int(round(yp))

def move_waypoint(actions, start_pos=(10, 1)):
    """
    >>> move_waypoint('''
    ...     F10
    ...     N3
    ...     F7
    ...     R90
    ...     F11
    ... ''')
    (214, -72)
    """
    sx, sy = 0, 0
    wx, wy = start_pos
    heading = 0  # degrees
    for action, amount in parse_actions(actions):
        if action == 'N':
            wy += amount
        elif action == 'S':
            wy -= amount
        elif action == 'E':
            wx += amount
        elif action == 'W':
            wx -= amount
        elif action == 'L':
            wx, wy = rotate(wx, wy, amount)
            heading = (heading + amount) % 360
        elif action == 'R':
            wx, wy = rotate(wx, wy, -amount)
            heading = (heading - amount) % 360
        elif action == 'F':
            sx += wx * amount
            sy += wy * amount
    return sx, sy

In [9]:
doctest.run_docstring_examples(rotate, globs=None, verbose=True)
doctest.run_docstring_examples(move_waypoint, globs=None, verbose=True)

Finding tests in NoName
Trying:
    rotate(10, 4, -90)
Expecting:
    (4, -10)
ok
Finding tests in NoName
Trying:
    move_waypoint('''
        F10
        N3
        F7
        R90
        F11
    ''')
Expecting:
    (214, -72)
ok


In [10]:
# Final answers
with open('day12.txt') as f:
    actions = f.read()
    print('Part 1: ', sum(abs(c) for c in move(actions)))
    print('Part 2: ', sum(abs(c) for c in move_waypoint(actions)))

Part 1:  1106
Part 2:  107281


# Day 13: Shuttle Search

In [188]:
def parse_schedule(text):
    """
    >>> parse_schedule('123\\nx,x,1,2,100,x')
    (123, ['x', 'x', 1, 2, 100, 'x'])
    """
    departure_ts, bus_ids = text.strip().split()
    departure_ts = int(departure_ts)
    bus_ids = [bus_id if is_wildcard(bus_id) else int(bus_id) for bus_id in bus_ids.split(',')]
    return departure_ts, bus_ids

def is_wildcard(bus_id):
    return bus_id == 'x'

def best_bus(ts, bus_ids):
    """
    >>> best_bus(939, [7, 13, 59, 'x', 31, 19])
    (59, 5)
    >>> best_bus(939, [7, 13, 59, 31, 19, 939])
    (939, 0)
    """
    bus_ids = [bus_id for bus_id in bus_ids if not is_wildcard(bus_id)]
    wait_time = lambda bus_id: (bus_id - (ts % bus_id)) % ts
    bus_id = min(bus_ids, key=wait_time)
    return bus_id, wait_time(bus_id)

In [183]:
doctest.run_docstring_examples(parse_schedule, globs=None, verbose=True)
doctest.run_docstring_examples(best_bus, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_schedule('123\nx,x,1,2,100,x')
Expecting:
    (123, ['x', 'x', 1, 2, 100, 'x'])
ok
Finding tests in NoName
Trying:
    best_bus(939, [7, 13, 59, 'x', 31, 19])
Expecting:
    (59, 5)
ok
Trying:
    best_bus(939, [7, 13, 59, 31, 19, 939])
Expecting:
    (939, 0)
ok


In [184]:
# Abandoned: too slow!
def divides(m, n):
    return n % m == 0

def make_factors(bus_ids):
    return {bus_id: i 
            for i, bus_id in enumerate(bus_ids) if not is_wildcard(bus_id)}

def is_solution(ts, factors, base_delta=0):
    return all(divides(f, ts + delta - base_delta) for f, delta in factors.items())

def earliest_timestamp(bus_ids, start=None):
    """
    >>> earliest_timestamp([7, 13, 'x', 'x', 59, 'x', 31, 19])
    1068781
    """
    factors = make_factors(bus_ids)
    max_ts = max(factors.keys())
    base_delta = factors[max_ts]
    guess = ((start // max_ts) + 1) * max_ts if start else max_ts
    while True:
        if is_solution(guess, factors, base_delta):
            return guess - base_delta
        guess += max_ts

In [185]:
doctest.run_docstring_examples(earliest_timestamp, globs=None, verbose=True)

Finding tests in NoName
Trying:
    earliest_timestamp([7, 13, 'x', 'x', 59, 'x', 31, 19])
Expecting:
    1068781
ok


In [235]:
@dataclass
class Solution:
    phase: int
    period: int

    @staticmethod
    def create(n, delta):
        return Solution((n - delta) % n, n)

def sync(s1, s2):
    """
    >>> sync(Solution.create(17, 0), Solution.create(1, 0))
    Solution(phase=0, period=17)
    >>> sync(sync(Solution.create(7, 0), Solution.create(13, 1)), Solution.create(59, 4)).phase
    350
    """
    # Only works if periods are prime numbers, I guess...
    hi = s1.period * s2.period
    for i in itertools.chain(range(s1.phase, hi, s1.period), range(s2.phase, hi, s2.period)):
        if (i % s1.period) == s1.phase and (i % s2.period) == s2.phase:
            return Solution(i, s1.period * s2.period)

def sync_all(bus_ids):
    """
    >>> sync_all([7, 13, 'x', 'x', 59, 'x', 31, 19])
    Solution(phase=1068781, period=3162341)
    """
    partials = [Solution.create(bus_id, i)
                for i, bus_id in enumerate(bus_ids) if not is_wildcard(bus_id)]
    return functools.reduce(sync, partials, Solution.create(1, 0))


In [236]:
doctest.run_docstring_examples(sync, globs=None, verbose=True)
doctest.run_docstring_examples(sync_all, globs=None, verbose=True)

Finding tests in NoName
Trying:
    sync(Solution.create(17, 0), Solution.create(1, 0))
Expecting:
    Solution(phase=0, period=17)
ok
Trying:
    sync(sync(Solution.create(7, 0), Solution.create(13, 1)), Solution.create(59, 4)).phase
Expecting:
    350
ok
Finding tests in NoName
Trying:
    sync_all([7, 13, 'x', 'x', 59, 'x', 31, 19])
Expecting:
    Solution(phase=1068781, period=3162341)
ok


In [237]:
%%time
# Final answers
with open('day13.txt') as f:
    ts, bus_ids = parse_schedule(f.read())
    bus_id, wait_time = best_bus(ts, bus_ids)
    print('Part 1: ', bus_id * wait_time)
    print('Part 2: ', sync_all(bus_ids).phase)

Part 1:  370
Part 2:  894954360381385
Wall time: 2 ms


# Day 14: Docking Data

In [69]:
def to_binary(x, width=36):
    """
    >>> to_binary(12, 4)
    '1100'
    >>> to_binary(12, 6)
    '001100'
    """
    return f'{x:0{width}b}'

def to_decimal(mask):
    return int(mask, 2)

def apply_mask(x, mask):
    """
    >>> apply_mask(11, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
    73
    >>> apply_mask(101, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
    101
    >>> apply_mask(0, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
    64
    """
    # Apply 1s.
    x |= to_decimal(mask.replace('X', '0'))
    # Apply 0s.
    x &= to_decimal(mask.replace('X', '1'))
    return x

def expand_mask(mask):
    """
    >>> list(expand_mask('11001'))
    ['11001']
    >>> list(expand_mask('X0X1'))
    ['0001', '0011', '1001', '1011']
    """
    wildcards = mask.count('X')
    for sample in range(2**wildcards):
        exp = mask
        for bit in to_binary(sample, wildcards):
            exp = exp.replace('X', bit, 1)
        yield exp


def expand_addr(addr, mask):
    """
    >>> list(expand_addr(42, '000000000000000000000000000000X1001X'))
    [26, 27, 58, 59]
    """
    masked_addr = ''.join(m if m in ('1', 'X') else a 
                          for a, m in zip(to_binary(addr), mask))
    for exp in expand_mask(masked_addr):
        yield to_decimal(exp)

def parse_program(program):
    for line in program:
        lhs, rhs = line.strip().split(' = ')
        if lhs == 'mask':
            yield lhs, rhs
        else:
            addr = int(lhs[lhs.index('[')+1:-1])
            val = int(rhs)
            yield lhs, (addr, val)
        

def run_program(program):
    """
    >>> program = ['mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X',
    ...            'mem[8] = 11', 'mem[7] = 101', 'mem[8] = 0']
    >>> run_program(program)
    165
    """
    mask = None
    mem = {}
    for opcode, operands in parse_program(program):
        if opcode == 'mask':
            mask = operands
        else:
            addr, val = operands
            mem[addr] = apply_mask(val, mask)
    return sum(mem.values())

def run_program_v2(program):
    """
    >>> program = ['mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X',
    ...            'mem[8] = 11', 'mem[7] = 101', 'mem[8] = 0']
    >>> run_program_v2(program)
    208
    """
    mask = None
    mem = {}
    for opcode, operands in parse_program(program):
        if opcode == 'mask':
            mask = operands
        else:
            addr, val = operands
            for addr in expand_addr(addr, mask):
                mem[addr] = val
    return sum(mem.values())

In [72]:
doctest.run_docstring_examples(to_binary, globs=None, verbose=True)
doctest.run_docstring_examples(apply_mask, globs=None, verbose=True)
doctest.run_docstring_examples(expand_mask, globs=None, verbose=True)
doctest.run_docstring_examples(expand_addr, globs=None, verbose=True)
doctest.run_docstring_examples(run_program, globs=None, verbose=True)
doctest.run_docstring_examples(run_program_v2, globs=None, verbose=True)

Finding tests in NoName
Trying:
    to_binary(12, 4)
Expecting:
    '1100'
ok
Trying:
    to_binary(12, 6)
Expecting:
    '001100'
ok
Finding tests in NoName
Trying:
    apply_mask(11, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
Expecting:
    73
ok
Trying:
    apply_mask(101, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
Expecting:
    101
ok
Trying:
    apply_mask(0, 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')
Expecting:
    64
ok
Finding tests in NoName
Trying:
    list(expand_mask('11001'))
Expecting:
    ['11001']
ok
Trying:
    list(expand_mask('X0X1'))
Expecting:
    ['0001', '0011', '1001', '1011']
ok
Finding tests in NoName
Trying:
    list(expand_addr(42, '000000000000000000000000000000X1001X'))
Expecting:
    [26, 27, 58, 59]
ok
Finding tests in NoName
Trying:
    program = ['mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X',
               'mem[8] = 11', 'mem[7] = 101', 'mem[8] = 0']
Expecting nothing
ok
Trying:
    run_program(program)
Expecting:
    165
ok
Finding tests in NoName
Trying:
  

KeyboardInterrupt: 

In [77]:
%%time
# Final answers
with open('day14.txt') as f:
    program = f.readlines()
    print('Part 1: ', run_program(program))
    print('Part 2: ', run_program_v2(program))

Part 1:  14553106347726
Part 2:  2737766154126
Wall time: 239 ms
